# KAN Visuals

## Imports

In [2]:
import kan

In [3]:
import kan
from sympy.printing import latex
from sympy import *
import sympy
from kan import KAN
from kan import MultKAN
import matplotlib.pyplot as plt
import torch
import numpy as np
import pandas as pd
import yaml
import os
from torchvision import transforms
from kan.spline import B_batch
import numpy as np
import importlib
importlib.reload(kan)

<module 'kan' from 'c:\\Users\\a829727\\OneDrive - Eviden\\Dokumente\\Uni\\Semester 6\\AML\\KAN-project\\KAN-Model\\kan\\__init__.py'>

In [4]:
os.makedirs('/pictures', exist_ok=True)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


## Data transform

In [5]:
# read train and test data
train = pd.read_csv('../data/MNIST/mnist_train.csv')
test = pd.read_csv('../data/MNIST/mnist_test.csv')
# convert data to numpy arrays
x_train = train.drop('label', axis=1).values
x_test = test.drop('label', axis=1).values
y_train = train['label'].values
y_test = test['label'].values

# set target size to scale images
target_size = 2

# define transformation
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((target_size, target_size)),
    transforms.ToTensor()
])

# transform the data
def transform_images(images):
    transformed_images = []
    for image in images:
        image = image.reshape(28, 28).astype(np.uint8)
        image = transform(image)
        image = image.view(-1)
        transformed_images.append(image)
    return torch.stack(transformed_images)

train_input = transform_images(x_train)
test_input = transform_images(x_test)
train_label = torch.tensor(y_train, dtype=torch.int64)
test_label = torch.tensor(y_test, dtype=torch.int64)

dataset = {
    'train_input': train_input,
    'train_label': train_label,
    'test_input': test_input,
    'test_label': test_label
}

# Convert labels to Long tensors
dataset['train_label'] = dataset['train_label'].long()
dataset['test_label'] = dataset['test_label'].long()

dataset['train_input'] = train_input.to(device)
dataset['test_input'] = test_input.to(device)
dataset['train_label'] = train_label.to(device)
dataset['test_label'] = test_label.to(device)

## Show first prprocessed picture

## Loading Model

In [6]:
#Parameter
hidden_1 = 3
hidden_2 = 0
steps = 5
grid = 3
k = 3
seed = 42
input_dim = train_input.shape[ 1 ]   # Anzahl der Eingabefunktionen
output_dim = 10   # Anzahl der Klasse

#Count
if hidden_2>0:
    Parameter = (hidden_1*input_dim+hidden_1*hidden_2+hidden_2*output_dim)*(grid+k+2)
    print(f"Parameter count = {Parameter}")
else:
    Parameter = hidden_1*(input_dim+output_dim)*(grid+k+2)
    print(f"Parameter count = {Parameter}")

Parameter count = 336


In [7]:
model = KAN(width=[input_dim, hidden_1, output_dim], grid= grid , k= k , seed= seed)  # Vereinfachte Modellkonfiguratio5

checkpoint directory created: ./model
saving model version 0.0


In [10]:
model = model.loadckpt("./Modelle/mini/model_mnist_h_3_s_5_g_3_k_3")

Aktuelles Verzeichnis: c:\Users\a829727\OneDrive - Eviden\Dokumente\Uni\Semester 6\AML\KAN-project\KAN-Model


## Show Model

In [11]:
def adjust_dict_colors(input_dict):
    """
    Passt jedes Value in einem Dictionary an, indem es zwischen blau (0) und rot (1) interpoliert.
    """
    def interpolate_color(value):
        """
        Interpoliert zwischen blau (0) und rot (1).
        """
        assert 0 <= value <= 1, "Value must be between 0 and 1"

        # RGB-Werte
        blue = np.array([0, 0, 1])  # blau
        red = np.array([1, 0, 0])   # rot

        # Lineare Interpolation
        color = blue * (1 - value) + red * value

        return color

    return {k: interpolate_color(v) for k, v in input_dict.items()}

def normalice(x):
    # Berechne den minimalen und maximalen Wert
    x_min = x.min()
    x_max = x.max()

    # Überprüfe, ob min und max gleich sind
    if x_min == x_max:
        x_normalized = torch.zeros_like(x)  # oder eine andere geeignete Konstante
    else:
        # Normalisiere den Tensor zwischen 0 und 1
        x_normalized = (x - x_min) / (x_max - x_min)

    return x_normalized
def forward_cursed(self, x, singularity_avoiding=False, y_th=10.):
        assert x.shape[1] == self.width_in[0]

        x = x[:, self.input_id.long()]

        # cache data
        self.cache_data = x

        self.acts = []  # shape ([batch, n0], [batch, n1], ..., [batch, n_L])
        self.acts_premult = []
        self.spline_preacts = []
        self.spline_postsplines = []
        self.spline_postacts = []
        self.acts_scale = []
        self.acts_scale_spline = []
        self.subnode_actscale = []
        self.edge_actscale = []
        self.postsplines = []

        self.acts.append(x)  # acts shape: (batch, width[l])

        for l in range(self.depth):
            print(f"Layer {l + 1} starts")  # Debugging
            acts_layer = []
            x_numerical, preacts, postacts_numerical, postspline = self.act_fun[l](x)
            self.postsplines.append(postspline)
            print(f"preacts Activation (Layer {l + 1}):: {preacts}")
            print(f"Postspline Activation (Layer {l + 1}):: {postspline}")

            if self.symbolic_enabled:
                x_symbolic, postacts_symbolic = self.symbolic_fun[l](x, singularity_avoiding=singularity_avoiding, y_th=y_th)
            else:
                x_symbolic = 0.
                postacts_symbolic = 0.

            x = x_numerical + x_symbolic

            if self.save_act:
                # save subnode_scale
                self.subnode_actscale.append(torch.std(x, dim=0).detach())

            # subnode affine transform
            x = self.subnode_scale[l][None,:] * x + self.subnode_bias[l][None,:]

            if self.save_act:
                postacts = postacts_numerical + postacts_symbolic
                # self.neurons_scale.append(torch.mean(torch.abs(x), dim=0))
                #grid_reshape = self.act_fun[l].grid.reshape(self.width_out[l + 1], self.width_in[l], -1)
                input_range = torch.std(preacts, dim=0) + 0.1
                output_range_spline = torch.std(postacts_numerical, dim=0) # for training, only penalize the spline part
                output_range = torch.std(postacts, dim=0) # for visualization, include the contribution from both spline + symbolic
                # save edge_scale
                self.edge_actscale.append(output_range)

                self.acts_scale.append((output_range / input_range).detach())
                self.acts_scale_spline.append(output_range_spline / input_range)
                self.spline_preacts.append(preacts.detach())
                self.spline_postacts.append(postacts.detach())
                self.spline_postsplines.append(postspline.detach())

                self.acts_premult.append(x.detach())

            # multiplication
            dim_sum = self.width[l+1][0]
            dim_mult = self.width[l+1][1]

            if self.mult_homo == True:
                for i in range(self.mult_arity-1):
                    if i == 0:
                        x_mult = x[:,dim_sum::self.mult_arity] * x[:,dim_sum+1::self.mult_arity]
                    else:
                        x_mult = x_mult * x[:,dim_sum+i+1::self.mult_arity]

            else:
                for j in range(dim_mult):
                    acml_id = dim_sum + np.sum(self.mult_arity[l+1][:j])
                    for i in range(self.mult_arity[l+1][j]-1):
                        if i == 0:
                            x_mult_j = x[:,[acml_id]] * x[:,[acml_id+1]]
                        else:
                            x_mult_j = x_mult_j * x[:,[acml_id+i+1]]

                    if j == 0:
                        x_mult = x_mult_j
                    else:
                        x_mult = torch.cat([x_mult, x_mult_j], dim=1)

            if self.width[l+1][1] > 0:
                x = torch.cat([x[:,:dim_sum], x_mult], dim=1)

            # x = x + self.biases[l].weight
            # node affine transform


            x = self.node_scale[l][None,:] * x + self.node_bias[l][None,:]

            self.acts.append(x.detach())

        # Dictionary initialisieren
        act_dict = {}

        # Iterieren über die Elemente des Tensors und hinzufügen zum Dictionary
        for l in range(len(self.postsplines)):
          self.postsplines[l][0] = normalice(self.postsplines[l][0])
          for i in range(self.postsplines[l][0].T.size(0)):
              for j in range(self.postsplines[l][0].T.size(1)):
                  act_dict[(l, i, j)] = self.postsplines[l][0].T[i, j].item()
        act_dict = adjust_dict_colors(act_dict)
        return x, act_dict

MultKAN.forward = forward_cursed

In [ ]:
# Originaler Tensor
original_tensor = dataset['test_input'][:1]

# Wiederhole den Tensor 5-mal entlang einer neuen Dimension
repeated_tensor = original_tensor.repeat(2, 1)

In [ ]:
original_tensor

Erste Layer immer 1 eintrag von tupel 0 zweiter eintrag des tupels

In [ ]:
out, act_dict_test = model(repeated_tensor)


In [ ]:
tensor([[ 0.2424, -0.0944, -0.0864,  0.2031],
        [ 0.1175,  0.3287,  0.2239, -0.7869],
        [-0.0983,  0.2654,  0.2650,  0.0174]]
0.2424 hat Key (0,0,0)
 0.1175 hat keey (0,0,1)
-0.0983 hat Key (0,0,2)
-0.0944 hat Key (0,1,0)
-0.0864 hat Key (0,1,1)
 0.2031 hat Key (0,1,2)

In [ ]:
first_act = postsplines
first_act

In [ ]:
len(postsplines)

In [ ]:
postsplines[0][0]

In [ ]:
postsplines[0][0].size(0)

In [ ]:
postsplines[0][0].size(1)

In [ ]:
postsplines[0][0][1, 0]

In [ ]:
# Dictionary initialisieren
act_dict = {}

# Iterieren über die Elemente des Tensors und hinzufügen zum Dictionary
for l in range(len(postsplines)):
  postsplines[l][0] = normalice(postsplines[l][0])
  for i in range(postsplines[l][0].T.size(0)):
      for j in range(postsplines[l][0].T.size(1)):
          act_dict[(l, i, j)] = postsplines[l][0].T[i, j].item()
act_dict = adjust_dict_colors(act_dict)

In [ ]:
act_dict_test

In [ ]:
act_dict

In [ ]:
edge_actscale

In [ ]:
spline_postsplines

In [ ]:
acts_premult

In [ ]:
out

In [ ]:
original_tensor

In [ ]:
inputs = acts[0][0]

In [ ]:
layer_1 = acts[1][0]

In [ ]:
layer_1

In [ ]:
layer_1_norm = normalice(layer_1)
colors_layer_1 = [interpolate_color(float(value)) for value in layer_1_norm]

In [ ]:
# Plotten des Farbverlaufs
fig, ax = plt.subplots(figsize=(10, 2))
for i, color in enumerate(colors):
    ax.axvline(x=i, color=color, linewidth=4)

ax.set_xticks([])
ax.set_yticks([])
plt.show()

In [ ]:
outputs = acts[2][0]
output_norm = normalice(outputs)


In [ ]:
output_norm

In [ ]:
max_index_out = int(torch.argmax(output_norm))